**<h1>Group Project</h1>**

**<h1>Lyrics to singer classification</h1>**
---

> 

   **<h2>Մասնակիցներ</h2>**


*   Հրանուշ Սահրադյան
*   Հասմիկ Մնացականյան
*   Մերի Վարդանյան
*   Արմեն Մանուկյան


>






**<h2>Purpose of the project</h2>**

Նախագծի նպատակն է համացանցից հավաքել (web scraping-ի միջոցով) 20 տարբեր հայ երգիչների երգերի բառեր, դրանք մշակելով ստանալ տվյալների բազմություն, բաժանել մի քանի մասի (train, val, test) և այդ տվյալների բազմության վրա ստանալ այնպիսի մոդել, որի միջոցով հնարավոր կլինի ինչ֊որ ճշտությամբ գուշակել, թե որ երգիչներին են պատկանում մոդելի կողմից չտեսնված երգերի բառերը։











**<h2>Contents</h2>**


1.   [Data Cleaning and Preprocessing](#preproc)
3.   [Feature Selection/Extraction](#fenigine)
4.   [EDA](#eda)


**Note**: Data collection part you can see in the notebook "Data collection"

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
import random
import nltk
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
import pickle
from datetime import datetime

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import time 
import glob
import requests
import nltk
from langdetect import detect_langs

In [2]:
import sys
sys.path.append('Data preprocessing')  # Add the folder to the Python module search path
import simple_feature_engineering as sf

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


 <a name="preproc"></a>

## Data Cleaning and Preprocessing

Data cleaning and preprocessing for a dataset on Armenian artists and their song lyrics typically involve several steps to ensure the data is accurate, consistent, and ready for analysis or modeling. Here's an overview of the potential steps involved:

**Handling Missing Values:** Identify any missing values in the dataset, such as empty cells or null values. Depending on the context, you can choose to either remove rows or columns with missing values, or impute the missing values using techniques like mean, median, or mode imputation. Consider the impact of missing values on variables like artist names, song titles, and lyrics, and handle them accordingly.

**Removing Duplicate Entries:** Check for and remove any duplicate entries in the dataset. Duplicates can arise from data collection or merging processes and can introduce bias or affect analysis results. Identify duplicate rows or combinations of variables and remove them to maintain data integrity.

**Cleaning and Standardizing Textual Data:** Perform text cleaning and preprocessing on variables like artist names, song titles, and lyrics. This may involve steps such as removing special characters, punctuation, or unnecessary whitespace, converting text to lowercase or uppercase consistently, and handling encoding issues if any.

**Language Detection:** Since you are interested in Armenian lyrics, you can apply language detection techniques to identify whether the lyrics are in Armenian or another language. This step ensures that you are working with the desired language and can filter out any non-Armenian lyrics if needed.

**Dealing with Stop Words and Noise:** In the lyrics column, consider removing common stop words (e.g., "the," "and," "is") that do not carry substantial meaning. Additionally, address any noise or irrelevant text that may be present in the lyrics, such as ad-libs or non-lyrical portions.

**Tokenization and Lemmatization:** Split the lyrics into individual words or tokens using tokenization techniques. This allows for further analysis or modeling on a word-level basis. Additionally, consider applying lemmatization to reduce words to their base or root form, improving consistency and reducing dimensionality.

**Handling Non-Textual Data:** If your dataset contains non-textual data, such as artist metadata or genre information, ensure appropriate cleaning and preprocessing. This may involve transforming categorical variables using techniques like one-hot encoding or label encoding to make them compatible with analysis or modeling techniques.

**Data Validation and Sanity Checks:** Validate the dataset against predefined constraints or business rules to ensure data integrity. Perform sanity checks to identify any inconsistencies, outliers, or unexpected patterns. This step helps ensure the dataset aligns with domain knowledge and meets your expectations.


In [17]:
df=pd.read_csv("Datasets/FinalData.csv")

Let's see what we have in our dataset.

In [18]:
df.shape

(1038, 3)

In [19]:
df.groupby('Artist').count()

,Song,Lyrics
Artist,,
Armenchik,26,26
Lav Eli,20,20
Aram MP3,32,32
Arame,38,38
Arsen Safaryan,28,28
Empyray,22,22
Forsh,22,22
Gor Sujyan,10,10
HT Hayko,32,32


In [20]:
print(df['Lyrics'].iloc[1033])


[Verse 1: Ֆելո (3.33)]

Մեզ թվումա, որ մեզ թվումա

Որ մեկը մեզ հետևումա, մեկը մեր հետևումա

Մեր մտքերը գողանումա, տանում

Ես լավանում եմ, դու՝ վատանում

Մենք որոշում ենք, հետո փոշմանում

Ասում ենք երկար ու երկար չենք անում

Ու եթե մի 10 հոգի էս պահին ներս գային

Լավ կլիներ մեր ռիթմերը զգային

Էս պահին, տես գահին

Մեջը կորած ասեղ, եթե գտնես քեզ պահի

Մատդ ծակի կասես, հանրագիտարանը բաց

Ու «Ա» տառից սկսի էս պահին

Ինչ կա շատ, հավասար կիսի

Անցնում էր լիմուզինը, դե սպանի

Մի գնդակով 2 ճագար չես սպանի

Ու եթե հոգի կա օդում մի քանի

Ուրեմն Էս պահին արթնացի մի քնի


[Hook: Ռոլանդ (Ռեինկարնացիա)]

Էս պահին չեմ ուզում էլ նայեմ ժամին

Ինձ թողեք մենակ ես սիրում եմ քամին

Ու ոչ մեկ չգիտի, որ քամին գույն ունի

Ու նայում է հեռվից էս պահին

Էս պահին չեմ ուզում էլ նայեմ ժամին

Ինձ թողեք մենակ ես սիրում եմ քամին

Ու ոչ մեկ չգիտի, որ քամին գույն ունի

Ու նայում է հեռվից էս պահին



It looks like the texts are formated in different styles: the first example includes text in round brackets which says how often parts of the text shall be repeated e.g. "(repeat previous line 3 times)".

Looks like the text in round brackets does mostly not belong to the lyrics of the song and should be removed.

In [21]:
text_in_round_brackets = sum(list(df['Lyrics'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
print('Number of round brackets: {}'.format(len(text_in_round_brackets)))

Number of round brackets: 1177


In [22]:
random.seed(0)
random.choices(text_in_round_brackets, k=20)

['Տուն 2',
 'Կրկներգ',
 'Կրկներգ',
 'Անցում',
 '3',
 'Կրկներգ',
 'Անցում',
 'Կրկներգ',
 'Տուն 1',
 'Աշխարհից քեզ թաքցրել է',
 'x2',
 'Բարև',
 'Կրկներգ',
 'Կրկներգ',
 'Ser',
 'Տուն 1',
 'x2',
 '2x',
 'Куплет 1',
 'Տուն 1']

Removeing round/square brackest and text within.

In [23]:
df['Lyrics'] = df['Lyrics'].map(lambda s: re.sub(r'\((.*?)\)', '', s))
df['Lyrics'] = df['Lyrics'].map(lambda s: re.sub(r'\[(.*?)\]', '', s))

As we can see all the songs written in Armenian within our dataset, so let's just remove any non-Armenian words present in the lyrics.

In [24]:
armenian_pattern = re.compile(r'[Ա-Ֆա-ֆև\s]+')
df['Lyrics'] = df['Lyrics'].apply(lambda x: ''.join(armenian_pattern.findall(x)))

df = df[df['Lyrics'].str.strip().astype(bool)]

df.reset_index(drop=True, inplace=True)


Let's get rid of duplicate songs too.

In [25]:
duplicates = df.duplicated(subset='Song')

# Check if there are any duplicates
if duplicates.any():
    # Remove the duplicate rows from the DataFrame
    df = df.drop_duplicates(subset='Song', keep='first').reset_index(drop=True)
    print("Duplicate values removed.")
else:
    print("No duplicate values found in the 'Song' column.")


Duplicate values removed.


In order to classify the 20 Armenian artists with the most songs in our dataset, we will keep only the records of those 20 artists in our dataframe.

In [26]:
artist_counts = df.groupby('Artist').count()
top_artists = artist_counts.nlargest(20, 'Song').index
df = df[df['Artist'].isin(top_artists)].reset_index(drop=True)

What we have after removing non armenian and duplicate songs for top 20 Artists in our dataset.

In [28]:
df.groupby('Artist').count()

,Song,Lyrics
Artist,,
Lav Eli,10,10
Aram MP3,18,18
Arame,11,11
Arsen Safaryan,24,24
Empyray,11,11
HT Hayko,16,16
Harout Pamboukjian,40,40
Mi Qani Hogi,16,16
Misho,29,29


In [29]:
df.shape

(543, 3)

<a name="fenigine"> </a>

## Feature Selection/Extraction

### Translating the obtained data to English 

In [3]:
df=pd.read_csv("Datasets/FinalData1.csv")

In [4]:
df=df.dropna()

In [5]:
df,ruben_songs=sf.delete_songs_by_artist(df,"ՌՈՒԲԵՆ ՀԱԽՎԵՐԴՅԱՆ",30)
df,komitas_songs=sf.delete_songs_by_artist(df,"ԿՈՄԻՏԱՍ",15)

In [6]:
df.groupby('Artist').count()

,Song,Lyrics,Translated_Lyrics
Artist,,,
Lav Eli,9,9,9
Aram MP3,17,17,17
Arame,9,9,9
Arsen Safaryan,23,23,23
Empyray,10,10,10
HT Hayko,16,16,16
Harout Pamboukjian,32,32,32
Mi Qani Hogi,15,15,15
Misho,26,26,26


In [74]:
df=sf.number_of_lines(df,'Lyrics')
df=sf.remove_new_lines(df,'Translated_Lyrics')
df=sf.remove_new_lines(df,'Translated_Lyrics')
df[['lines','Lyrics']]

,lines,Lyrics
0,104,\r\n\tԱհա և վերջ\n\r\n\tՎերջ հարաժամ և հարատև\...
1,68,\r\n\tՄեծածավալ և հիասքանչ տաճարների\n\r\n\tԵվ...
2,78,\r\n\tԵրբ անառակ որդու նման\n\r\n\tԴառնաս թողա...
3,48,\r\n\tՍիրտս կարյունի աշխարհի ցավով\n\r\n\tՈրքա...
4,54,\r\n\tԱռաջ գիտեի\n\r\n\tԹե մարդիկ ինչու կգային...
...,...,...
537,74,\n\n\nՄի Քանի Հոգի\n\n\n\n\nՔեզ տանող այս ճանա...
538,55,\n\n\nՄեզ թվումա որ մեզ թվումա\n\nՈր մեկը մեզ ...
539,53,\n\n\nՄի Քանի Հոգի\n\nՄՔՀ\n\n\n\n\nԳտածդ ալիք ...
541,82,\n\n\n մեկը \n\nԴուրը տեղը \n\nՄեկը մ...


#### Tokenization
A very common way to analyse text is to seperate it into a list of words which makes it much easier to do further analysis. I'm using nltk.tokenize to do that. Furthermore, all punctuations are removed as well. Below is an example how the text is converted.

In [75]:
df=sf.tokenize_column(df,'Lyrics','hy')
df=sf.tokenize_column(df,'Translated_Lyrics')
df[['Song','tokens_hy']]

,Song,tokens_hy
0,ԱՀԱ ԵՎ ՎԵՐՋ,"[Ահա, և, վերջ, Վերջ, հարաժամ, և, հարատև, Իբրև,..."
1,ԱՄԵՆԸ ՁԵԶ,"[Մեծածավալ, և, հիասքանչ, տաճարների, Եվ, ոսկեզա..."
2,ԱՆԱՌԱԿ ՈՐԴՈՒ ՎԵՐԱԴԱՐՁԸ,"[Երբ, անառակ, որդու, նման, Դառնաս, թողած, քաղա..."
3,ԱՇԽԱՐՀԻ ՑԱՎՈՎ,"[Սիրտս, կարյունի, աշխարհի, ցավով, Որքան, տառապ..."
4,ԱՌԱՋ ԳԻՏԵԻ,"[Առաջ, գիտեի, Թե, մարդիկ, ինչու, կգային, աշխար..."
...,...,...
537,Chanapar,"[Մի, Քանի, Հոգի, Քեզ, տանող, այս, ճանապարհը, ո..."
538,Es Pahin,"[Մեզ, թվումա, որ, մեզ, թվումա, Որ, մեկը, մեզ, ..."
539,Rhyme,"[Մի, Քանի, Հոգի, ՄՔՀ, Գտածդ, ալիք, էր, ծովը, չ..."
541,Yes Qo Nmanei,"[մեկը, Դուրը, տեղը, Մեկը, մեկը, բոլորը, Մեկը, ..."


In [76]:
sf.lowercase_column(df,'tokens_hy')
sf.lowercase_column(df,'tokens_en')

df[['tokens_hy','tokens_en']]

,tokens_hy,tokens_en
0,"[ահա, և, վերջ, վերջ, հարաժամ, և, հարատև, իբրև,...","[here, is, an, end, end, of, paragraphs, and, ..."
1,"[մեծածավալ, և, հիասքանչ, տաճարների, եվ, ոսկեզա...","[large, scale, and, wonderful, temples, and, t..."
2,"[երբ, անառակ, որդու, նման, դառնաս, թողած, քաղա...","[when, like, a, prodigal, son, become, your, o..."
3,"[սիրտս, կարյունի, աշխարհի, ցավով, որքան, տառապ...","[my, heart, with, kariy, s, world, pain, how, ..."
4,"[առաջ, գիտեի, թե, մարդիկ, ինչու, կգային, աշխար...","[i, was, known, before, what, people, would, b..."
...,...,...
537,"[մի, քանի, հոգի, քեզ, տանող, այս, ճանապարհը, ո...","[several, people, what, will, this, way, to, y..."
538,"[մեզ, թվումա, որ, մեզ, թվումա, որ, մեկը, մեզ, ...","[it, seems, to, us, seems, to, be, that, one, ..."
539,"[մի, քանի, հոգի, մքհ, գտածդ, ալիք, էր, ծովը, չ...","[several, people, icf, did, you, find, a, wave..."
541,"[մեկը, դուրը, տեղը, մեկը, մեկը, բոլորը, մեկը, ...","[one, like, one, one, all, one, one, holiday, ..."


Now we can remove stop words from tokens.

In [77]:
sf.remove_stopwords(df,'tokens_en')
print(df[['Song','tokens_en']])

                       Song                                          tokens_en
0               ԱՀԱ ԵՎ ՎԵՐՋ  [end, end, paragraphs, lasting, last, closed, ...
1                 ԱՄԵՆԸ ՁԵԶ  [large, scale, wonderful, temples, prayer, gol...
2    ԱՆԱՌԱԿ ՈՐԴՈՒ ՎԵՐԱԴԱՐՁԸ  [like, prodigal, son, become, city, seek, door...
3             ԱՇԽԱՐՀԻ ՑԱՎՈՎ  [heart, kariy, world, pain, much, suffering, g...
4                ԱՌԱՋ ԳԻՏԵԻ  [known, people, would, world, sun, would, pour...
..                      ...                                                ...
537                Chanapar  [several, people, way, perhaps, ask, one, day,...
538                Es Pahin  [seems, us, seems, one, watching, us, one, tho...
539                   Rhyme  [several, people, icf, find, wave, sea, scream...
541           Yes Qo Nmanei  [one, like, one, one, one, one, holiday, home,...
542             Lav Jamanak  [refresh, lucky, enough, side, employe, well, ...

[456 rows x 2 columns]


In [78]:
sf.remove_stopwords(df,'tokens_hy','hy')
print(df[['Song','tokens_hy']])

                       Song                                          tokens_hy
0               ԱՀԱ ԵՎ ՎԵՐՋ  [վերջ, վերջ, հարաժամ, հարատև, իբրև, վերջին, փա...
1                 ԱՄԵՆԸ ՁԵԶ  [մեծածավալ, հիասքանչ, տաճարների, եվ, ոսկեզարդ,...
2    ԱՆԱՌԱԿ ՈՐԴՈՒ ՎԵՐԱԴԱՐՁԸ  [երբ, անառակ, որդու, նման, դառնաս, թողած, քաղա...
3             ԱՇԽԱՐՀԻ ՑԱՎՈՎ  [սիրտս, կարյունի, աշխարհի, ցավով, որքան, տառապ...
4                ԱՌԱՋ ԳԻՏԵԻ  [առաջ, գիտեի, մարդիկ, ինչու, կգային, աշխարհ, ա...
..                      ...                                                ...
537                Chanapar  [քանի, հոգի, քեզ, տանող, ճանապարհը, ուր, կհասց...
538                Es Pahin  [մեզ, թվումա, մեզ, թվումա, մեկը, մեզ, հետևումա...
539                   Rhyme  [քանի, հոգի, մքհ, գտածդ, ալիք, էր, ծովը, չար, ...
541           Yes Qo Nmanei  [մեկը, դուրը, տեղը, մեկը, մեկը, բոլորը, մեկը, ...
542             Lav Jamanak  [թարմացրու, քեզ, իմացի, բախտը, քո, կողքինա, էղ...

[456 rows x 2 columns]


#### Stemming
Stemming is a simple method to convert words to a common base form. For example converting plural to singular or past tense to present tense. This helps to treat words with the same meaning in the same way.

In contrast to lemmatization it uses a very simple method and does not always find the grammatically correct from. However, as stemming is much faster than lemmatization I'm using it here.

Below is an example how different forms of the word "make" are converted. In this case stemming failes to correctly convert the past tense.

In [79]:
df=sf.map_tokens_to_stems(df,'tokens_en')

In [80]:
df=sf.calculate_stem_word_count(df,'stems_en')
print(df[['Song','stems_en','n_stems_en']])

                       Song  \
0               ԱՀԱ ԵՎ ՎԵՐՋ   
1                 ԱՄԵՆԸ ՁԵԶ   
2    ԱՆԱՌԱԿ ՈՐԴՈՒ ՎԵՐԱԴԱՐՁԸ   
3             ԱՇԽԱՐՀԻ ՑԱՎՈՎ   
4                ԱՌԱՋ ԳԻՏԵԻ   
..                      ...   
537                Chanapar   
538                Es Pahin   
539                   Rhyme   
541           Yes Qo Nmanei   
542             Lav Jamanak   

                                              stems_en  n_stems_en  
0    [end, end, paragraph, last, last, close, page,...         128  
1    [larg, scale, wonder, templ, prayer, golden, t...          84  
2    [like, prodig, son, becom, citi, seek, door, w...          76  
3    [heart, kariy, world, pain, much, suffer, god,...          71  
4    [known, peopl, would, world, sun, would, pour,...          69  
..                                                 ...         ...  
537  [sever, peopl, way, perhap, ask, one, day, ele...         100  
538  [seem, us, seem, one, watch, us, one, thought,...          81  
539  [

In [81]:
df['n_tokens_hy'] = df['tokens_hy'].map(len)

In this section we have used TFIDF to analyse word frequencies in more detail. TFIDF means term frequency - inverse document frequency. Term frequency means how often a word appears in a specific text (in this case song lyrics). Inverse document frequency is the inverse frequency of the same word in the whole document (in this case all song lyrics).

The idea of TFIDF is to find out if a specific word appears unusually often in the text. If this is the case, this word gets a high value, if the word appears more often in other texts, its value will be low.

This method is very common to cluster texts with similar topics.

In [82]:
cv = CountVectorizer()

# generate word counts
df['stems_en_str'] = df['stems_en'].apply(lambda tokens: ' '.join(tokens))
stem_count_vector = cv.fit_transform(df['stems_en_str'])

# compute idf
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(stem_count_vector)

TfidfTransformer()

Below are the words with the lowest values. These words apear many times in almost each song. Thus, these words are not usefull to characterise a song.

In [83]:
tfidf_df = pd.DataFrame({'stem': cv.get_feature_names(), 'weight': tfidf_transformer.idf_})
 
# get lowest weights
tfidf_df.sort_values('weight').head()

C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



,stem,weight
2798,love,1.607230
2035,heart,1.826366
2717,life,1.959897
3354,one,2.037087
2727,like,2.043279


In [84]:
# get highest weights
tfidf_df.sort_values('weight', ascending=False).head()

,stem,weight
0,aa,6.431536
3085,monday,6.431536
3063,miz,6.431536
3064,mke,6.431536
3065,mko,6.431536


On the other hand, these are words with the highest values.

Next we calculate an TFIDF vector for each song. Every element of this vector represents one word. Its value is calculated by multiplying its frequency with the corresponding weight which was calculated above. This value is normalised by the total number of words of the song. Words which don't appear in the text get a value of 0.

The TFIDF score is the sum of the TFIDF vector elements. The higher the score the more unusal the words of this song are compared to other songs.

In [85]:
# assign tf idf scores to each song
tf_idf_vector = tfidf_transformer.transform(stem_count_vector)

# attach count vectors to dataframe
tf_idf_vector_lst = [-1] * len(df)
for i in range(len(df)):
    tf_idf_vector_lst[i] = tf_idf_vector[i]
df['tf_idf_vector'] = tf_idf_vector_lst    

df['tf_idf_score'] = df['tf_idf_vector'].map(lambda vec: np.sum(vec.todense()))

In [86]:
df[['Song','tf_idf_score']]

,Song,tf_idf_score
0,ԱՀԱ ԵՎ ՎԵՐՋ,6.881928
1,ԱՄԵՆԸ ՁԵԶ,6.928256
2,ԱՆԱՌԱԿ ՈՐԴՈՒ ՎԵՐԱԴԱՐՁԸ,5.222807
3,ԱՇԽԱՐՀԻ ՑԱՎՈՎ,7.544911
4,ԱՌԱՋ ԳԻՏԵԻ,5.694038
...,...,...
537,Chanapar,6.015289
538,Es Pahin,5.527543
539,Rhyme,7.093200
541,Yes Qo Nmanei,3.614634


In [87]:
fig = px.box(df, x='Artist', y='tf_idf_score', title='TFIDF scores of songs per artist')
fig.show()

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer

def calculate_tf_idf(df, column_name, artist_column):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df[column_name].apply(' '.join))
    feature_names = vectorizer.get_feature_names_out()
    scores = tfidf_matrix.toarray()
    artists = df[artist_column]
    data = {'Word': [], 'Artist': [], 'TF-IDF': []}
    for i, word in enumerate(feature_names):
        for j, artist in enumerate(artists):
            data['Word'].append(word)
            data['Artist'].append(artist)
            data['TF-IDF'].append(scores[j][i])
    return pd.DataFrame(data)
tf_idf_data = calculate_tf_idf(df, 'stems_en', 'Artist')


In [89]:
tf_idf_data

,Word,Artist,TF-IDF
0,aa,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,0.0
1,aa,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,0.0
2,aa,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,0.0
3,aa,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,0.0
4,aa,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,0.0
...,...,...,...
2436859,zurna,Mi Qani Hogi,0.0
2436860,zurna,Mi Qani Hogi,0.0
2436861,zurna,Mi Qani Hogi,0.0
2436862,zurna,Mi Qani Hogi,0.0


In [90]:
df.stems_en

0      [end, end, paragraph, last, last, close, page,...
1      [larg, scale, wonder, templ, prayer, golden, t...
2      [like, prodig, son, becom, citi, seek, door, w...
3      [heart, kariy, world, pain, much, suffer, god,...
4      [known, peopl, would, world, sun, would, pour,...
                             ...                        
537    [sever, peopl, way, perhap, ask, one, day, ele...
538    [seem, us, seem, one, watch, us, one, thought,...
539    [sever, peopl, icf, find, wave, sea, scream, p...
541    [one, like, one, one, one, one, holiday, home,...
542    [refresh, lucki, enough, side, employ, well, a...
Name: stems_en, Length: 456, dtype: object

In [91]:
# caclculate mean vector
def get_mean_vector(vec_lst):
    return csr_matrix(vstack(vec_lst).mean(axis=0))

In [92]:
# calculate mean vector over all songs of same artist
artist_df = df.groupby('Artist').agg({'tf_idf_vector': get_mean_vector, 'Song': len}).reset_index()\
                   .rename(columns={'Song': 'songs'})

# get selected artists
artist_filter_df = artist_df.loc[artist_df['Artist'].isin(df['Artist'])]

In [93]:
similarity_matrix = cosine_similarity(vstack(artist_filter_df['tf_idf_vector']), 
                                      vstack(artist_filter_df['tf_idf_vector']))
artist_names = artist_filter_df['Artist'].tolist()
fig = go.Figure(data=go.Heatmap(z=np.flipud(similarity_matrix), x=artist_names, y=list(reversed(artist_names)), 
                                colorscale='balance', zmin=0.5, zmax=1.1))
fig.show()

This matrix visualises the similarity between TFIDF artist vectors. Again, a value of 1 (red) means that the vectors are identical which only appears when comparing vectors of the same artist. 

In [94]:
artist_song_filter_df = pd.merge(artist_filter_df[['Artist', 'tf_idf_vector', 'songs']].assign(key = 0), 
                                 df[['Artist', 'tf_idf_vector', 'Song']].assign(key = 0), on='key', 
                                 suffixes=['_artist', '_song']).drop('key', axis=1).reset_index(drop=True)
artist_song_filter_df['same_artist'] = artist_song_filter_df['Artist_artist'] == artist_song_filter_df['Artist_song']

In [95]:
artist_song_filter_df['vector_similarity'] = \
    artist_song_filter_df.apply(lambda row: tf_idf_vector_similarity(row['tf_idf_vector_artist'], 
                                                                     row['tf_idf_vector_song'], 
                                                                     row['songs'], row['same_artist']), axis=1)

NameError: name 'tf_idf_vector_similarity' is not defined

In [96]:
# calculate similarity of artist tf idf vector and song vector
def tf_idf_vector_similarity(artist_vector, song_vector, songs, same_artist):
    # check if song is from same artist
    if same_artist:
        # deduct song vector from artist vector
        artist_vector = (songs * artist_vector - song_vector) / (songs - 1)
    # calculate similarity
    return cosine_similarity(artist_vector, song_vector)[0][0]

In [97]:
artist_song_filter_df

,Artist_artist,tf_idf_vector_artist,songs,Artist_song,tf_idf_vector_song,Song,same_artist
0,Lav Eli,"(0, 12)\t0.007865804986752812\n (0, 32)\t0....",9,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,"(0, 5123)\t0.11227978102984074\n (0, 5087)\...",ԱՀԱ ԵՎ ՎԵՐՋ,False
1,Lav Eli,"(0, 12)\t0.007865804986752812\n (0, 32)\t0....",9,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,"(0, 5246)\t0.4412385242123455\n (0, 5236)\t...",ԱՄԵՆԸ ՁԵԶ,False
2,Lav Eli,"(0, 12)\t0.007865804986752812\n (0, 32)\t0....",9,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,"(0, 5230)\t0.04574269091145943\n (0, 5199)\...",ԱՆԱՌԱԿ ՈՐԴՈՒ ՎԵՐԱԴԱՐՁԸ,False
3,Lav Eli,"(0, 12)\t0.007865804986752812\n (0, 32)\t0....",9,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,"(0, 5236)\t0.12427558296283031\n (0, 5234)\...",ԱՇԽԱՐՀԻ ՑԱՎՈՎ,False
4,Lav Eli,"(0, 12)\t0.007865804986752812\n (0, 32)\t0....",9,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,"(0, 5244)\t0.31779844055718776\n (0, 5236)\...",ԱՌԱՋ ԳԻՏԵԻ,False
...,...,...,...,...,...,...,...
9115,Սիրուշո,"(0, 15)\t0.009929488432933562\n (0, 18)\t0....",25,Mi Qani Hogi,"(0, 5230)\t0.12717770501276457\n (0, 5215)\...",Chanapar,False
9116,Սիրուշո,"(0, 15)\t0.009929488432933562\n (0, 18)\t0....",25,Mi Qani Hogi,"(0, 5244)\t0.05969598557033705\n (0, 5239)\...",Es Pahin,False
9117,Սիրուշո,"(0, 15)\t0.009929488432933562\n (0, 18)\t0....",25,Mi Qani Hogi,"(0, 5216)\t0.061932050332675265\n (0, 5179)...",Rhyme,False
9118,Սիրուշո,"(0, 15)\t0.009929488432933562\n (0, 18)\t0....",25,Mi Qani Hogi,"(0, 5223)\t0.3019773221844423\n (0, 4658)\t...",Yes Qo Nmanei,False


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 506 entries, 0 to 542
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Artist             506 non-null    object 
 1   Song               506 non-null    object 
 2   Lyrics             506 non-null    object 
 3   Translated_Lyrics  506 non-null    object 
 4   lines              506 non-null    int64  
 5   tokens_hy          506 non-null    object 
 6   tokens_en          506 non-null    object 
 7   stems_en           506 non-null    object 
 8   n_stems_en         506 non-null    int64  
 9   n_tokens_hy        506 non-null    int64  
 10  stems_en_str       506 non-null    object 
 11  tf_idf_vector      506 non-null    object 
 12  tf_idf_score       506 non-null    float64
dtypes: float64(1), int64(3), object(9)
memory usage: 55.3+ KB


<a name="eda"></a>
# EDA
#### Number of songs, artists, and words

In [27]:
song_count_df = df.groupby('Artist')[['Song']].count()
fig = px.histogram(song_count_df, x='Song', title='Songs per artist', labels={'Song': 'Songs'})
fig.show()

In [28]:
# words per song
fig = px.histogram(df, x='n_stems_en', title='Words per song')
fig.show()

In [29]:
from plotly.subplots import make_subplots

# Create two subplots
fig = make_subplots(rows=1, cols=2)

# Plot for n_tokens_en
fig.add_trace(
    go.Histogram(x=df['n_stems_en'], nbinsx=20, name='English'),
    row=1, col=1
)

# Plot for n_tokens_hy
fig.add_trace(
    go.Histogram(x=df['n_tokens_hy'], nbinsx=20, name='Armenian'),
    row=1, col=2
)

# Update layout
fig.update_layout(
    title='Words per Song',
    xaxis=dict(title='Number of Words'),
    yaxis=dict(title='Count'),
    barmode='overlay'
)

# Show the plot
fig.show()


In [30]:
fig = px.box(df, x='Artist', y='n_tokens_hy', title='Word count(hy) per song by artist')
fig.show()

Next I check the ratio of unique words over all words. This ratio is 1 if all words of a song are different, the more repeated words appearing in a song the lower this ratio becomes.

In [31]:
df['n_unique_stems_en'] = df['stems_en'].map(lambda lst: len(set(lst)))

df['unique_stems_ratio_en'] = df['n_unique_stems_en'] / df['n_stems_en']

df['n_unique_tokens_hy'] = df['tokens_hy'].map(lambda lst: len(set(lst)))

df['unique_tokens_ratio_hy'] = df['n_unique_tokens_hy'] / df['n_tokens_hy']

In [32]:
fig = px.box(df, x='Artist', y='unique_stems_ratio_en', title='Ratio of unique words to all words')
fig.show()

In [33]:
fig = px.box(df, x='Artist', y='unique_tokens_ratio_hy', title='Ratio of unique words to all words(hy)')
fig.show()

Adding sentiment of each song as new feature.
The TextBlob library is used to calculate the sentiment polarity of each song's translated lyrics. The sentiment polarity ranges from -1 to 1, where values closer to 1 indicate a positive sentiment and values closer to -1 indicate a negative sentiment.

In [34]:
df['sentiment'] = df['Translated_Lyrics'].apply(lambda text: TextBlob(text).sentiment.polarity)


print(df[['Song','sentiment']])

                       Song  sentiment
0               ԱՀԱ ԵՎ ՎԵՐՋ  -0.193333
1                 ԱՄԵՆԸ ՁԵԶ   0.258333
2    ԱՆԱՌԱԿ ՈՐԴՈՒ ՎԵՐԱԴԱՐՁԸ  -0.012500
3             ԱՇԽԱՐՀԻ ՑԱՎՈՎ   0.060173
4                ԱՌԱՋ ԳԻՏԵԻ   0.118182
..                      ...        ...
537                Chanapar  -0.016875
538                Es Pahin   0.133333
539                   Rhyme   0.060014
541           Yes Qo Nmanei  -0.187500
542             Lav Jamanak   0.306000

[506 rows x 2 columns]


In [35]:
fig = px.box(df, x='Artist', y='sentiment', title='Sentiment by artist')
fig.show()

Now our dataset is ready for models.

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 506 entries, 0 to 542
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Artist                  506 non-null    object 
 1   Song                    506 non-null    object 
 2   Lyrics                  506 non-null    object 
 3   Translated_Lyrics       506 non-null    object 
 4   lines                   506 non-null    int64  
 5   tokens_hy               506 non-null    object 
 6   tokens_en               506 non-null    object 
 7   stems_en                506 non-null    object 
 8   n_stems_en              506 non-null    int64  
 9   n_tokens_hy             506 non-null    int64  
 10  stems_en_str            506 non-null    object 
 11  tf_idf_vector           506 non-null    object 
 12  tf_idf_score            506 non-null    float64
 13  n_unique_stems_en       506 non-null    int64  
 14  unique_stems_ratio_en   506 non-null    fl

In [ ]:
feature = ['n_stems_diff', 'n_stems_diff_std',
       'unique_stems_ratio_diff', 'unique_stems_ratio_diff_std',
       'stems_per_line_diff', 'stems_per_line_diff_std', 'tf_idf_score_diff',
       'tf_idf_score_diff_std', 'polarity_diff', 'polarity_diff_std',
       'vector_similarity']
df_lst = []
for f in feature:
    df = artist_song_feature['train'][['same_artist']]
    df['feature'] = f
    df['value'] = artist_song_feature['train'][f]
    df_lst.append(df)
feature_df = pd.concat(df_lst)
feature_df.head()

<a name="models"></a>

# Models

Train/Test split.

In [37]:
from sklearn.model_selection import train_test_split

features = df[['n_stems_en', 'n_tokens_hy', 'n_unique_stems_en', 'unique_stems_ratio_en', 'n_unique_tokens_hy', 'unique_tokens_ratio_hy', 'sentiment', 'tf_idf_score']]
target = df['Artist']

# Split the data into train and test sets
train_X, test_X, train_y, test_y = train_test_split(features, target, test_size=0.1, random_state=42)

print("Train set shape:", train_X.shape, train_y.shape)
print("Test set shape:", test_X.shape, test_y.shape)

Train set shape: (404, 8) (404,)
Test set shape: (102, 8) (102,)


### SVM

In [40]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [41]:
label_encoder = LabelEncoder()
train_y_encoded = label_encoder.fit_transform(train_y)
test_y_encoded = label_encoder.transform(test_y)

We are going to use grid search to find best parameters for SVM classifier.

In [43]:
# Define the SVM classifier
svm = SVC()

# Define the hyperparameters for grid search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf','poly']    
}

grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(train_X, train_y_encoded)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

best_model.fit(train_X, train_y_encoded)

# Predict the target variable for the test set
predictions_encoded = best_model.predict(test_X)
predictions = label_encoder.inverse_transform(predictions_encoded)

# Calculate the accuracy of the model
accuracy = accuracy_score(test_y, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.28431372549019607


In [44]:
report = classification_report(test_y, predictions, output_dict=True)
report_df = pd.DataFrame(report).transpose()
print("Classification Report:")
print(report_df)

Classification Report:
                    precision    recall  f1-score     support
Aram MP3             0.000000  0.000000  0.000000    4.000000
Arame                0.000000  0.000000  0.000000    2.000000
Arsen Safaryan       0.000000  0.000000  0.000000    4.000000
Empyray              0.000000  0.000000  0.000000    2.000000
HT Hayko             0.750000  1.000000  0.857143    3.000000
Harout Pamboukjian   0.000000  0.000000  0.000000    8.000000
Mi Qani Hogi         0.250000  0.200000  0.222222    5.000000
Misho                0.666667  0.250000  0.363636    8.000000
Razmik Amyan         0.000000  0.000000  0.000000    3.000000
Silva Hakobyan       0.000000  0.000000  0.000000    2.000000
ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ       0.176471  0.300000  0.222222   10.000000
Անդրե                0.000000  0.000000  0.000000    3.000000
Լիլիթ Հովհաննիսյան   0.166667  0.333333  0.222222    3.000000
ԿՈՄԻՏԱՍ              0.285714  0.750000  0.413793    8.000000
Հայկ Հակոբյան        0.200000  0.166667  0.1818

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

